In [1]:
import sys
from pathlib import Path

ROOT = Path().resolve().parent
sys.path.append(str(ROOT))

In [2]:
import torchvision.transforms as transforms
from data_preparation.data_preparation import grs_prepare_data_loaders, visualize_dataset,visualize_class_distribution
from models.models_created import CNNLSTM
from torchinfo import summary
import torch
from livelossplot import PlotLosses
import torch.nn as nn

In [3]:
PATH_OSATS = "../data/raw/OSATS.xlsx"
PATH_TRAIN = "../data/processed/"
BATCH_SIZE = 2

In [4]:
transforms = transforms.Compose([
    transforms.CenterCrop(760),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(  # Normalize using ImageNet statistics
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [5]:
train_dl, val_dl, test_dl, train_dl_all, val_dl_all, test_dl_all = grs_prepare_data_loaders(0.70, PATH_TRAIN, PATH_OSATS,transforms=transforms, batch_size=BATCH_SIZE)

In [6]:
visualize_dataset(train_dl, test_dl, val_dl)

Casos de Treino: 42
Casos de Validação: 6
Casos de Teste: 12
(TREINO) Shape tensor batch -> input: torch.Size([2, 600, 3, 224, 224]), output: torch.Size([2])
(VAL) Shape tensor batch -> input: torch.Size([2, 600, 3, 224, 224]), output: torch.Size([2])
(TESTE) Shape tensor batch -> input: torch.Size([2, 600, 3, 224, 224]), output: torch.Size([2])
Valor maximo:2.640000104904175 Valor mínimo:-2.1179039478302


In [7]:
visualize_class_distribution(train_dl_all, test_dl_all,val_dl_all,plot=False)

helllo
Distribuição de classes (GRS):
Novice       | Train:  15 | Val:   2 | Test:   4
Intermediate | Train:   9 | Val:   1 | Test:   2
Proficient   | Train:  15 | Val:   1 | Test:   4
Specialist   | Train:   1 | Val:   1 | Test:   2


In [7]:
model = CNNLSTM()
print(summary(model, input_size=(BATCH_SIZE, 4, 3,224,224), verbose=0))

Layer (type:depth-idx)                        Output Shape              Param #
CNNLSTM                                       [2, 4]                    --
├─Sequential: 1-1                             [8, 512, 1, 1]            --
│    └─Conv2d: 2-1                            [8, 64, 112, 112]         9,408
│    └─BatchNorm2d: 2-2                       [8, 64, 112, 112]         128
│    └─ReLU: 2-3                              [8, 64, 112, 112]         --
│    └─MaxPool2d: 2-4                         [8, 64, 56, 56]           --
│    └─Sequential: 2-5                        [8, 64, 56, 56]           --
│    │    └─BasicBlock: 3-1                   [8, 64, 56, 56]           73,984
│    │    └─BasicBlock: 3-2                   [8, 64, 56, 56]           73,984
│    └─Sequential: 2-6                        [8, 128, 28, 28]          --
│    │    └─BasicBlock: 3-3                   [8, 128, 28, 28]          230,144
│    │    └─BasicBlock: 3-4                   [8, 128, 28, 28]          295,42

In [8]:

def train_model(h5_file, train_dl, val_dl, model, criterion, optimizer,device='cpu'):
    liveloss = PlotLosses()
    
    for epoch in range(EPOCHS):
        logs = {}

        # ---------- Treino ----------
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_dl:
            inputs = inputs.to(device)        # [B, T, C, H, W]
            labels = labels.to(device)        # [B]

            outputs = model(inputs)           # [B, num_classes]
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_corrects += (preds == labels).sum().item()

        epoch_loss = running_loss / len(train_dl.dataset)
        epoch_acc = running_corrects / len(train_dl.dataset)

        logs['loss'] = epoch_loss
        logs['accuracy'] = epoch_acc

        # ---------- Validação ----------
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in val_dl:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_corrects += (preds == labels).sum().item()

        epoch_val_loss = val_loss / len(val_dl.dataset)
        epoch_val_acc = val_corrects / len(val_dl.dataset)

        logs['val_loss'] = epoch_val_loss
        logs['val_accuracy'] = epoch_val_acc

        liveloss.update(logs)
        liveloss.send()

    torch.save(model, h5_file)

In [9]:
EPOCHS = 20
CRITERION = nn.CrossEntropyLoss()
LEARNING_RATE = 0.001
OPTIMIZER = torch.optim.Adam(model.parameters(), LEARNING_RATE)

In [ ]:
train_model(
    h5_file="../models/CNNLSTM.pth",
    train_dl=train_dl,
    val_dl=val_dl,
    model=model,
    criterion=CRITERION,
    optimizer=OPTIMIZER
)